In [39]:
# !pip install torch-summary

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import pandas as pd
import os
# from pl_bolts.models.autoencoders import VAE
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [21]:
# os.getcwd()
root_dir = "/workspace/vae-fairness/"
data_dir = os.path.join(root_dir,"data")
raw_data_dir = os.path.join(data_dir,"raw")
raw_german_all = os.path.join(raw_data_dir,"german.data")
raw_bar_pass = os.path.join(raw_data_dir,"bar_pass_prediction.csv")
# raw_german_all

In [22]:
bar_df = pd.read_csv(raw_bar_pass).set_index('ID')
bar_df

,decile1b,decile3,decile1,sex,race,cluster,lsat,ugpa,zfygpa,DOB_yr,grad,zgpa,bar1,bar1_yr,bar2,bar2_yr,fulltime,fam_inc,age,gender,parttime,male,race1,race2,Dropout,other,asian,black,hisp,pass_bar,bar,bar_passed,tier,index6040,indxgrp,indxgrp2,dnn_bar_pass_prediction,gpa
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,10.0,10.0,10.0,1.0,7.0,1.0,44.0,3.5,1.33,69.0,Y,1.88,P,7.0,P,94.0,1.0,5.0,-62.0,female,0.0,0.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,4.0,886.842082,g 700+,i 820+,0.979804,3.5
3,5.0,4.0,5.0,1.0,7.0,2.0,29.0,3.5,-0.11,69.0,Y,-0.57,P,7.0,P,94.0,1.0,4.0,-62.0,female,0.0,0.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,2.0,649.999987,f 640-700,f 640-700,0.979804,3.5
36,3.0,2.0,3.0,2.0,7.0,3.0,36.0,3.5,-0.64,65.0,Y,-1.12,P,7.0,P,94.0,1.0,1.0,-58.0,male,0.0,1.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,3.0,760.526298,g 700+,h 760-820,0.979804,3.5
52,7.0,4.0,7.0,2.0,7.0,3.0,39.0,3.5,0.34,58.0,Y,-0.49,P,7.0,P,94.0,1.0,4.0,-51.0,male,0.0,1.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,3.0,807.894717,g 700+,h 760-820,0.979804,3.5
55,9.0,8.0,9.0,2.0,7.0,4.0,48.0,3.5,1.02,68.0,Y,0.76,P,7.0,P,94.0,1.0,4.0,-61.0,male,0.0,1.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,5.0,949.999974,g 700+,i 820+,0.979804,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10695,3.0,1.0,3.0,2.0,3.0,6.0,26.5,1.8,-0.58,64.0,Y,-1.49,F,2.0,F,95.0,1.0,2.0,-62.0,male,0.0,1.0,black,a black,NO,0,0,1,0,0,c Failed,False,1.0,440.526304,b 400-460,b 400-460,0.557568,1.8
19642,2.0,1.0,3.0,2.0,3.0,6.0,19.7,1.8,-0.64,64.0,Y,-1.36,F,7.0,F,94.0,1.0,3.0,-57.0,male,0.0,1.0,black,a black,NO,0,0,1,0,0,c Failed,False,1.0,333.157888,a under 400,a under 400,0.557568,1.8
24704,7.0,8.0,7.0,2.0,3.0,3.0,36.0,1.8,0.41,66.0,Y,0.80,P,7.0,P,95.0,2.0,3.0,-59.0,male,1.0,1.0,black,a black,NO,0,0,1,0,1,a Passed 1st time,True,3.0,590.526298,e 580-640,e 580-640,0.557568,1.8


In [23]:

bar_df['age'] = bar_df['age'].abs()

drop_cols_dup = ['DOB_yr','sex','bar2','race2','other','asian','black','hisp','bar','bar_passed','indxgrp','ugpa','indxgrp2','gender','race1'] #,'bar2_yr'
drop_cols_na = ['decile1b','decile3','decile1','zgpa','zfygpa']
one_hot_cols = ['grad','bar1','race','Dropout']
bar_df = bar_df.drop(columns = drop_cols_dup + drop_cols_na)
bar_df = bar_df.dropna()
bar_df = pd.get_dummies(bar_df,columns = one_hot_cols)
# sens_attri = ['other','asian','black','hisp', 'male','race1','race2', 'sex','race','DOB_yr','age','gender']

In [24]:
X_train, X_test = train_test_split(bar_df, test_size=0.2, random_state=42)
scaler = MinMaxScaler()#StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [25]:
# X_train_scaled = pd.DataFrame(X_train_scaled,columns = X_train.columns,index = X_train.index)

In [26]:
X_train.shape

(17515, 29)

In [28]:
# torch.tensor(X_train_scaled.values,dtype=torch.float32)#.shape
X_train_torch = torch.tensor(X_train_scaled,dtype=torch.float32)#.shape
X_test_torch = torch.tensor(X_test_scaled,dtype=torch.float32)#.shape
trainloader=DataLoader(dataset=X_train_torch,batch_size=1024)
testloader=DataLoader(dataset=X_test_torch,batch_size=1024)


In [29]:
class Autoencoder(nn.Module):
    def __init__(self,D_in,H = 10, H2 = 12, latent_dim = 3):
        super(Autoencoder,self).__init__()
        # Encoder
        self.lin1 = nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.lin2 = nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features = H2)
        self.lin3 = nn.Linear(H2,H2)
        self.lin_bn3 = nn.BatchNorm1d(num_features = H2)
        
        # Latent
        self.fc1 = nn.Linear(H2,latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features = latent_dim)
        self.fc21 = nn.Linear(latent_dim,latent_dim)
        self.fc22 = nn.Linear(latent_dim,latent_dim)
        
        #Sampling vector
        self.fc3 = nn.Linear(latent_dim,latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim,H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)
        
        # Decoder
        self.lin4 = nn.Linear(H2,H2)
        self.lin_bn4 = nn.BatchNorm1d(num_features = H2)
        self.lin5 = nn.Linear(H2,H)
        self.lin_bn5 = nn.BatchNorm1d(num_features = H)
        self.lin6 = nn.Linear(H,D_in)
        self.lin_bn6 = nn.BatchNorm1d(num_features = D_in)
        self.relu = nn.ReLU()
        
    def encode(self, x):
        linear1 = self.relu(self.lin_bn1(self.lin1(x)))
        linear2 = self.relu(self.lin_bn2(self.lin2(linear1)))
        linear3 = self.relu(self.lin_bn2(self.lin2(linear2)))
        
        fc1 = F.relu(self.bn1(self.fc1(linear3)))
        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)
        
        return r1,r2
    
    def reparameterize(self,mu,logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
        
    def decode(self,z):
        fc3 = self.relu(self.fc_bn3(self.fc3(z)))
        fc4 = self.relu(self.fc_bn4(self.fc4(fc3)))
        
        linear4 = self.relu(self.lin_bn4(self.lin4(fc4)))
        linear5 = self.relu(self.lin_bn5(self.lin5(linear4)))
        return self.lin_bn6(self.lin6(linear5))
    
    def forward(self,x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu,logvar)
        return self.decode(z), mu, logvar
    
class customLoss(nn.Module):
    def __init__(self):
        super(customLoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")
    
    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD

In [32]:
D_in = X_train_torch.shape[1]
H = 50
H2 = 12
model = Autoencoder(D_in, H, H2).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_mse = customLoss()

In [44]:
X_train_torch[0].shape

torch.Size([29])

In [47]:
trainloader.dataset.shape

torch.Size([17515, 29])

In [48]:
summary(model, (1, 29))

RuntimeError: Failed to run torchsummary. See above stack traces for more details. Executed layers up to: [Linear: 1-1]

In [33]:
epochs = 1500
log_interval = 50
val_losses = []
train_losses = []
test_losses = []

In [34]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    if epoch % 200 == 0:        
        print('====> Epoch: {} Average training loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        train_losses.append(train_loss / len(trainloader.dataset))

In [35]:
def test(epoch):
    with torch.no_grad():
        test_loss = 0
        for batch_idx, data in enumerate(testloader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            loss = loss_mse(recon_batch, data, mu, logvar)
            test_loss += loss.item()
            if epoch % 200 == 0:        
                print('====> Epoch: {} Average test loss: {:.4f}'.format(
                    epoch, test_loss / len(testloader.dataset)))
            test_losses.append(test_loss / len(testloader.dataset))

In [36]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1024x12 and 50x12)

In [19]:
bar_df.parttime.value_counts()

0.0    20653
1.0     1720
Name: parttime, dtype: int64

In [28]:
bar_df.Dropout.value_counts()

NO     21893
YES        1
Name: Dropout, dtype: int64

In [25]:
bar_df.dropna().pass_bar.value_counts(normalize=True)
# bar_df.dropna().isna().sum()

1    0.948707
0    0.051293
Name: pass_bar, dtype: float64

In [76]:
bar_df['bar'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%'

a Passed 1st time    88.54%
b Passed 2nd time     6.24%
c Failed              5.22%
e non-Grad             0.0%
Name: bar, dtype: object

In [48]:
# bar_df = bar_df.fillna(0)
bar_df.groupby(['bar','bar_passed']).size().unstack(fill_value=0)


bar_passed,False,True
bar,,
a Passed 1st time,0,19839
b Passed 2nd time,0,1398
c Failed,1169,0
e non-Grad,1,0


In [58]:
bar_df.groupby(['indxgrp','indxgrp2']).size().unstack(fill_value=0)


indxgrp2,a under 400,b 400-460,c 460-520,d 520-580,e 580-640,f 640-700,g 700-760,h 760-820,i 820+
indxgrp,,,,,,,,,
a under 400,39,0,0,0,0,0,0,0,0
b 400-460,0,129,0,0,0,0,0,0,0
c 460-520,0,0,407,0,0,0,0,0,0
d 520-580,0,0,0,829,0,0,0,0,0
e 580-640,0,0,0,0,1950,0,0,0,0
f 640-700,0,0,0,0,0,3781,0,0,0
g 700+,0,0,0,0,0,0,5129,4902,5241


In [68]:
bar_df.groupby(['race','hisp','other','asian','black']).size().unstack(fill_value=0)#.reset_index()#.reset_index()


,,,black,0,1
race,hisp,other,asian,,
1.0,1,1,0,105,0
2.0,0,0,1,897,0
3.0,0,0,0,0,1343
4.0,0,0,0,396,0
5.0,0,0,0,125,0
6.0,0,0,0,506,0
7.0,0,0,0,18716,0
8.0,0,1,0,303,0


In [51]:
bar_df.groupby('race')['dnn_bar_pass_prediction'].mean()

race
1.0    0.964075
2.0    0.981285
3.0    0.965381
4.0    0.974287
5.0    0.976062
6.0    0.979271
7.0    0.982041
8.0    0.981207
Name: dnn_bar_pass_prediction, dtype: float64

In [62]:
bar_df.groupby('gender')['dnn_bar_pass_prediction'].mean()

gender
female    0.983048
male      0.978830
Name: dnn_bar_pass_prediction, dtype: float64

In [80]:
pass_by_age = bar_df.groupby(['age','bar_passed'],dropna = False).size().unstack(fill_value=0)
pass_by_age['pass_rate'] = pass_by_age[True] / (pass_by_age[True]+pass_by_age[False])
pass_by_age.sort_values('age',ascending = False)

bar_passed,False,True,pass_rate
age,,,
-3.0,0,1,1.000000
-5.0,0,3,1.000000
-7.0,0,1,1.000000
-8.0,0,2,1.000000
-9.0,0,1,1.000000
-13.0,0,1,1.000000
-17.0,0,1,1.000000
-18.0,0,1,1.000000
-21.0,1,1,0.500000


In [86]:
pass_by_race = bar_df.groupby(['race1','bar_passed'],dropna = False).size().unstack(fill_value=0)
pass_by_race['total'] = (pass_by_race[True]+pass_by_race[False])
pass_by_race['pass_rate'] = pass_by_race[True] / pass_by_race['total']
pass_by_race['proportion_total'] = pass_by_race['total']/pass_by_race['total'].sum()
pass_by_race['proportion_failed'] = pass_by_race[False]/pass_by_race[False].sum()#*100
pass_by_race['normalized_failure_likelihood'] = pass_by_race['proportion_failed']/pass_by_race['proportion_total']

pass_by_race.sort_values('pass_rate')

bar_passed,False,True,total,pass_rate,proportion_total,proportion_failed,normalized_failure_likelihood
race1,,,,,,,
black,298,1045,1343,0.778109,0.059937,0.254701,4.249503
NaN,2,14,16,0.875000,0.000714,0.001709,2.393910
hisp,128,899,1027,0.875365,0.045834,0.109402,2.386917
other,42,366,408,0.897059,0.018209,0.035897,1.971456
asian,70,827,897,0.921962,0.040032,0.059829,1.494526
white,630,18086,18716,0.966339,0.835275,0.538462,0.644652


In [71]:
pass_by_gender = bar_df.groupby(['gender','bar_passed'],dropna = False).size().unstack(fill_value=0)
pass_by_gender['pass_rate'] = pass_by_gender[True] / (pass_by_gender[True]+pass_by_gender[False])
pass_by_gender.sort_values('pass_rate')

bar_passed,False,True,pass_rate
gender,,,
female,569,9257,0.942092
male,601,11975,0.952211
NaN,0,5,1.000000


In [49]:
bar_df

,decile1b,decile3,decile1,sex,race,cluster,lsat,ugpa,zfygpa,DOB_yr,grad,zgpa,bar1,bar1_yr,bar2,bar2_yr,fulltime,fam_inc,age,gender,parttime,male,race1,race2,Dropout,other,asian,black,hisp,pass_bar,bar,bar_passed,tier,index6040,indxgrp,indxgrp2,dnn_bar_pass_prediction,gpa
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,10.0,10.0,10.0,1.0,7.0,1.0,44.0,3.5,1.33,69.0,Y,1.88,P,7.0,P,94.0,1.0,5.0,-62.0,female,0.0,0.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,4.0,886.842082,g 700+,i 820+,0.979804,3.5
3,5.0,4.0,5.0,1.0,7.0,2.0,29.0,3.5,-0.11,69.0,Y,-0.57,P,7.0,P,94.0,1.0,4.0,-62.0,female,0.0,0.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,2.0,649.999987,f 640-700,f 640-700,0.979804,3.5
36,3.0,2.0,3.0,2.0,7.0,3.0,36.0,3.5,-0.64,65.0,Y,-1.12,P,7.0,P,94.0,1.0,1.0,-58.0,male,0.0,1.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,3.0,760.526298,g 700+,h 760-820,0.979804,3.5
52,7.0,4.0,7.0,2.0,7.0,3.0,39.0,3.5,0.34,58.0,Y,-0.49,P,7.0,P,94.0,1.0,4.0,-51.0,male,0.0,1.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,3.0,807.894717,g 700+,h 760-820,0.979804,3.5
55,9.0,8.0,9.0,2.0,7.0,4.0,48.0,3.5,1.02,68.0,Y,0.76,P,7.0,P,94.0,1.0,4.0,-61.0,male,0.0,1.0,white,b white,NO,0,0,0,0,1,a Passed 1st time,True,5.0,949.999974,g 700+,i 820+,0.979804,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10695,3.0,1.0,3.0,2.0,3.0,6.0,26.5,1.8,-0.58,64.0,Y,-1.49,F,2.0,F,95.0,1.0,2.0,-62.0,male,0.0,1.0,black,a black,NO,0,0,1,0,0,c Failed,False,1.0,440.526304,b 400-460,b 400-460,0.557568,1.8
19642,2.0,1.0,3.0,2.0,3.0,6.0,19.7,1.8,-0.64,64.0,Y,-1.36,F,7.0,F,94.0,1.0,3.0,-57.0,male,0.0,1.0,black,a black,NO,0,0,1,0,0,c Failed,False,1.0,333.157888,a under 400,a under 400,0.557568,1.8
24704,7.0,8.0,7.0,2.0,3.0,3.0,36.0,1.8,0.41,66.0,Y,0.80,P,7.0,P,95.0,2.0,3.0,-59.0,male,1.0,1.0,black,a black,NO,0,0,1,0,1,a Passed 1st time,True,3.0,590.526298,e 580-640,e 580-640,0.557568,1.8


In [21]:
pd.read_csv("/workspace/vae-fairness/data/raw/german.data-numeric", delimiter=",")#.shape
# pd.read_csv(german_all, delimiter="\t")
# pd.read_csv(raw_bar_pass)

,1 6 4 12 5 5 3 4 1 67 3 2 1 2 1 0 0 1 0 0 1 0 0 1 1
0,2 48 2 60 1 3 2 2 1 22 3 ...
1,4 12 4 21 1 4 3 3 1 49 3 ...
2,1 42 2 79 1 4 3 4 2 45 3 ...
3,1 24 3 49 1 3 3 4 4 53 3 ...
4,4 36 2 91 5 3 3 4 4 35 3 ...
...,...
994,4 12 2 17 1 4 2 4 1 31 3 ...
995,1 30 2 39 1 3 1 4 2 40 3 ...
996,4 12 2 8 1 5 3 4 3 38 3 ...
997,1 45 2 18 1 3 3 4 4 23 3 ...
